# **📄 Document type classification baseline code**
> 문서 타입 분류 대회에 오신 여러분 환영합니다! 🎉     
> 아래 baseline에서는 ResNet 모델을 로드하여, 모델을 학습 및 예측 파일 생성하는 프로세스에 대해 알아보겠습니다.

## Contents
- Prepare Environments
- Import Library & Define Functions
- Hyper-parameters
- Load Data
- Train Model
- Inference & Save File


## 1. Prepare Environments

* 데이터 로드를 위한 구글 드라이브를 마운트합니다.
* 필요한 라이브러리를 설치합니다.

In [1]:
# # 구글 드라이브 마운트, Colab을 이용하지 않는다면 패스해도 됩니다.
# from google.colab import drive
# drive.mount('/gdrive', force_remount=True)
# drive.mount('/content/drive')

Mounted at /gdrive
Mounted at /content/drive


In [2]:
# # 구글 드라이브에 업로드된 대회 데이터를 압축 해제하고 로컬에 저장합니다.
# !tar -xvf drive/MyDrive/datasets_fin.tar > /dev/null

In [9]:
!pip install timm
!pip install augraphy

In [7]:
!pip install opencv-python

In [11]:
import os
import pandas as pd
import numpy as np
import cv2
from PIL import Image
import albumentations as A
from albumentations.pytorch import ToTensorV2
from augraphy import *
import timm
import torch
import torch.nn as nn
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from torch.optim import NAdam
from tqdm import tqdm
from sklearn.metrics import f1_score
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## 2. Import Library & Define Functions
* 학습 및 추론에 필요한 라이브러리를 로드합니다.
* 학습 및 추론에 필요한 함수와 클래스를 정의합니다.

In [12]:
train_transform = A.Compose([
    A.Resize(height=512, width=512),
    A.HorizontalFlip(p=0.35),
    A.VerticalFlip(p=0.35),
    A.OneOf([A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=(15, 25)),A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=(-15, -25))], p=0.3),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
])
test_transform = A.Compose([
    A.Resize(height=512, width=512),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
])
train_augraphy = AugraphyPipeline([
BadPhotoCopy(),DirtyDrum(),NoiseTexturize(),Folding(), ShadowCast(shadow_opacity_range=(0,0.35)), LightingGradient(), LowLightNoise(p = 0.5)])

In [13]:
class ImageDataset(Dataset):
    def __init__(self, csv, path, transform=None, augraphy=None):
        self.df = pd.read_csv(csv).values
        self.path = path
        self.transform = transform
        self.augraphy = augraphy
    def __len__(self):
        return len(self.df)
    def __getitem__(self, idx):
        name, target = self.df[idx]
        img = np.array(Image.open(os.path.join(self.path, name)))
        if img.shape[1] > img.shape[0]:
            img = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
        if self.augraphy:
            img = self.augraphy(img)
        if self.transform:
            img = self.transform(image=img)['image']
        return img, target

model = timm.create_model('maxvit_xlarge_tf_512', pretrained=True, num_classes=17).to(device)
optimizer = NAdam(model.parameters(), lr=5e-5)
loss_fn = nn.CrossEntropyLoss()


model.safetensors:   0%|          | 0.00/1.90G [00:00<?, ?B/s]

In [15]:
for _ in range(30):
    model.train()
    train_dataset = ImageDataset("/data/ephemeral/home/data/train.csv", "/data/ephemeral/home/data/train", transform=train_transform, augraphy= train_augraphy)
    train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True, num_workers=4, pin_memory=True, drop_last=True)
    preds_list, targets_list = [], []
    for image, targets in tqdm(train_loader):
        image, targets = image.to(device), targets.to(device)
        optimizer.zero_grad()
        preds = model(image)
        loss = loss_fn(preds, targets)
        loss.backward()
        optimizer.step()
        preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())
        targets_list.extend(targets.detach().cpu().numpy())
    print(f1_score(targets_list, preds_list, average="macro"))


100%|██████████| 1570/1570 [09:02<00:00,  2.90it/s]


0.7155280309067065


100%|██████████| 1570/1570 [09:02<00:00,  2.89it/s]


0.8926140299387384


100%|██████████| 1570/1570 [09:03<00:00,  2.89it/s]


...

0.9864989375891287


100%|██████████| 1570/1570 [09:02<00:00,  2.89it/s]


0.9859858036290584


100%|██████████| 1570/1570 [09:02<00:00,  2.90it/s]

0.9914194631843429


In [16]:
test_dataset = ImageDataset("/data/ephemeral/home/data/sample_submission.csv", "/data/ephemeral/home/data/test", transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=0, pin_memory=True)
preds_list = []
model.eval()
with torch.no_grad():
    for image, _ in tqdm(test_loader):
        preds = model(image.to(device))
        preds_list.extend(preds.argmax(dim=1).cpu().numpy())
pd.DataFrame({'ID': test_dataset.df[:, 0], 'target': preds_list}).to_csv("sub.csv", index=False)

100%|██████████| 3140/3140 [08:23<00:00,  6.24it/s]
